In [78]:
import pandas as pd
df=pd.read_csv("C:\\cafe-sentiment-analytics\\data\\cafe_reviews_10000.csv")
df

,review_id,rating,review_text,review_date,reviewer_name,branch_name,category,sentiment_manual
0,1,4,Very average experience. Overpriced for the po...,6/21/2024,Rahul,"ChaiCofi, Kochi",NaN,NaN
1,2,5,Not worth the hype. Staff seemed uninterested.,6/3/2024,Saniya,"French Toast, Panampilly",NaN,NaN
2,3,3,Service was slow and the food was cold when se...,9/4/2024,Gokul,"Kashi Art Café, Fort Kochi",NaN,NaN
3,4,3,Loved the desserts here! Coffee was perfect.,12/28/2024,Joel,"Kashi Art Café, Fort Kochi",NaN,NaN
4,5,3,One of the best cafés in Kochi. Highly recomme...,9/7/2024,Merin,"Kashi Art Café, Fort Kochi",NaN,NaN
...,...,...,...,...,...,...,...,...
9995,9996,3,One of the best cafés in Kochi. Highly recomme...,1/3/2024,Joseph,"French Toast, Panampilly",NaN,NaN
9996,9997,4,One of the best cafés in Kochi. Highly recomme...,7/25/2024,Harish,"Bloomsbury's, Kakkanad",NaN,NaN
9997,9998,1,One of the best cafés in Kochi. Highly recomme...,10/25/2024,Akshay,"French Toast, Panampilly",NaN,NaN
9998,9999,3,Perfect spot for brunch. Peaceful and aesthetic.,11/12/2024,Merin,"ChaiCofi, Kochi",NaN,NaN


In [79]:
# creating ml model for sentiment analysis
def create_ml_label(rating):
    if rating >= 4:
        return "positive"
    elif rating <= 2:
        return "negative"
    else:
        return None

df["sentiment_ml"] = df["rating"].apply(create_ml_label)

In [80]:
df_ml=df.dropna(subset=["sentiment_ml"])

In [81]:
df_ml.shape

(7967, 9)

In [82]:
df_ml=df_ml[["review_text","sentiment_ml"]]
df_ml.head()

,review_text,sentiment_ml
0,Very average experience. Overpriced for the po...,positive
1,Not worth the hype. Staff seemed uninterested.,positive
5,Not worth the hype. Staff seemed uninterested.,negative
6,Not worth the hype. Staff seemed uninterested.,positive
7,Not worth the hype. Staff seemed uninterested.,negative


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
x = df_ml["review_text"]
y = df_ml["sentiment_ml"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [85]:
# creating TF-IDF Vectorizer
tfidf=TfidfVectorizer(
    lowercase=True,
    stop_words="english",
    max_features=5000,
    ngram_range=(1,2)
)

In [86]:
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)


In [87]:
# shapes
x_train_tfidf.shape, x_test_tfidf.shape

((6373, 87), (1594, 87))

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [89]:
model =LogisticRegression(
    max_iter=1000,
    random_state=42
)
model.fit(x_train_tfidf, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",42
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`multi

In [90]:
# making the prerdictions
y_pred = model.predict(x_test_tfidf)

In [91]:
# evaluate the performance
accuracy = model.score(x_test_tfidf, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5062735257214555


In [92]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.50      0.52      0.51       791
    positive       0.51      0.49      0.50       803

    accuracy                           0.51      1594
   macro avg       0.51      0.51      0.51      1594
weighted avg       0.51      0.51      0.51      1594



In [93]:
# confusion metrix
confusion_matrix(y_test,y_pred)

array([[412, 379],
       [408, 395]])

In [94]:
df_test = x_test.reset_index(drop=True).to_frame(name="review_text")
df_test["true_sentiment"] = y_test.reset_index(drop=True)
df_test["predicted_sentiment"] = y_pred

In [95]:
comparison = (
    df_test["true_sentiment"] == df_test["predicted_sentiment"]
).mean()

comparison

np.float64(0.5062735257214555)

In [96]:
# Inspect failures
df_test[df_test["true_sentiment"] != df_test["predicted_sentiment"]].head(10)

,review_text,true_sentiment,predicted_sentiment
3,Loved the desserts here! Coffee was perfect.,negative,positive
5,Service was slow and the food was cold when se...,negative,positive
7,Service was slow and the food was cold when se...,negative,positive
8,Very average experience. Overpriced for the po...,negative,positive
13,Crowded but food quality is really good.,negative,positive
17,Amazing place to chill. Strong coffee and good...,positive,negative
19,Perfect spot for brunch. Peaceful and aesthetic.,positive,negative
24,Great food and lovely ambience. Service was qu...,negative,positive
26,Amazing place to chill. Strong coffee and good...,positive,negative
29,Very average experience. Overpriced for the po...,negative,positive


In [97]:
# Save ML comparison results
df_test["ml_correct"] = (
    df_test["true_sentiment"] == df_test["predicted_sentiment"]
)

df_test.to_csv("../outputs/ml_predictions.csv", index=False)